# Project | Sequence Model in NLP

# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Sequence_Model/Fake News Challenge/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
train_bodies = pd.read_csv(project_path +'train_bodies.csv') 
train_stances = pd.read_csv(project_path +'train_stances.csv')   

In [6]:
train_bodies.head(1)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...


In [7]:
train_bodies.shape

(1683, 2)

In [8]:
train_stances.head(1)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated


In [9]:
train_stances.shape

(49972, 3)

In [0]:
dataset = train_bodies.merge(train_stances, how = 'inner' , on = "Body ID",)

In [11]:
dataset.shape

(49972, 4)


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [12]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


In [13]:
dataset.shape

(49972, 4)

## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t = Tokenizer(num_words=MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
t.fit_on_texts(dataset['articleBody'])
t.fit_on_texts(dataset['Headline'])

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [19]:
len(dataset['articleBody'])

49972

In [0]:
unique_id = list(dataset["Body ID"].unique())

In [21]:
len(dataset)

49972

In [0]:
from nltk.tokenize import sent_tokenize
texts = []
articles = []
for i in range(len(dataset) ):
  text= list(dataset['articleBody'])[i]
  sent = sent_tokenize(text)
  texts.append(text)
  articles.append(sent)

## Check 2:

first element of texts and articles should be as given below. 

In [23]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [24]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence
import numpy as np

In [0]:
shape = (len(articles), MAX_SENTS, MAX_SENT_LENGTH)
data = np.zeros(shape)

In [27]:
data.shape

(49972, 20, 20)

In [0]:
for i in range(len(articles)):
  for j in range(len(articles[i])):
    if j > 19:
      j=19
    a = [x for x, in [x for x in t.texts_to_sequences(text_to_word_sequence(articles[i][j])) if x ]]
    for k in range(len(a)):
      if k > 19:
        k=19
      data[i, j, k] = a[k]    

In [29]:
data.shape

(49972, 20, 20)

In [0]:
data = data.astype('int32')

### Check 3:

Accessing first element in data should give something like given below.

In [31]:
data[0, :, :]

array([[    3,   481,   427,  7211,    81,     3,  3733,   331,     5,
         3891,   350,     4,  1431,  2958,     1,    89,    12,   464,
            0,     0],
       [  758,    95,  1047,     3,  2679,  1752,     7,   189,     3,
         1217,  1075,  2030,   700,   159,     1,  3032,   448,     1,
          555,   235],
       [   89,  1067,  4115,  2349,    12,     3,  1092,  3306,    19,
            1,    89,     2,  1793,     1,   521,  2009,    15,     9,
            3,  3111],
       [  181,  3640,   972,   200,  2556,    44,  6775,  1722,  1252,
            5, 13317, 17936,     1,   778,    31,   740,  3990,    67,
           85,     0],
       [ 2349,    12,  1557,    38,  1094,   351,   775,     2,   367,
          260,  1770,     5,  4450,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   700,   189,    19,     1,   427,    32,     3,  7417,
            4,  2159,  1252,     6,     3,  5270,     4,  1217,  1252,
           12,  3363],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_heading = []
articles_heading = []
for i in range(len(dataset) ):
  text= list(dataset['Headline'])[i]
  sent = sent_tokenize(text)
  texts_heading.append(text)
  articles_heading.append(sent)

In [33]:
texts_heading[0]

'Soldier shot, Parliament locked down after gunfire erupts at war memorial'

In [34]:
articles_heading[0]

['Soldier shot, Parliament locked down after gunfire erupts at war memorial']

In [35]:
shape = (len(articles_heading), MAX_SENTS, MAX_SENT_LENGTH)
data_heading = np.zeros(shape)
data_heading.shape

(49972, 20, 20)

In [0]:
for i in range(len(articles_heading)):
  for j in range(len(articles_heading[i])):
    if j > 19:
      j=19
    h = [x for x, in [x for x in t.texts_to_sequences(text_to_word_sequence(articles_heading[i][j])) if x ]]
    for k in range(len(h)):
      if k > 19:
        k=19
      data_heading[i, j, k] = h[k]    

In [37]:
data_heading = data_heading.astype('int32')
data_heading.shape

(49972, 20, 20)

In [38]:
data_heading[-1, :, :]

array([[6105, 6363, 2165, 2157, 1856, 1998,   14,  284,  145, 2379,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = np.array(pd.get_dummies(dataset['Stance']))
labels = labels.astype('int32')

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [40]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split
x_train, x_val, x_heading_train, x_heading_val, y_train, y_val= train_test_split( data,data_heading,labels, test_size=0.2)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [44]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [54]:
vocab_size = len(t.word_index)
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from tensorflow.python.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Dropout, Dense, Embedding, Flatten, LSTM, Input, concatenate,Reshape
import tensorflow as tf
tf.keras.backend.clear_session()

### Model

In [74]:
input1 = Input(shape=(20,20,))
input2 = Input(shape=(20,20,))
reshape1 = Reshape((20*20,))(input1)
reshape2 = Reshape((20*20,))(input2)
Embedding1=Embedding(vocab_size, 100, input_length=MAX_SENT_LENGTH,
                   weights=[embedding_matrix],
                   trainable=False)(reshape1)
Embedding2=Embedding(vocab_size, 100, input_length=MAX_SENT_LENGTH,
                   weights=[embedding_matrix],
                   trainable=False)(reshape2)                   
lstm1=LSTM(100, dropout=0.2, recurrent_dropout=0.2)(Embedding1)
lstm2=LSTM(100, dropout=0.2, recurrent_dropout=0.2)(Embedding2)
concat=concatenate([lstm1,lstm2],axis=1)
dense=Dense(4,activation='softmax')(concat)
model=Model(inputs=[input1,input2],outputs=[dense])


### Compile and fit the model

In [75]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20)]     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 20, 20)]     0                                            
__________________________________________________________________________________________________
reshape (Reshape)               (None, 400)          0           input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 400)          0           input_2[0][0]                    
______________________________________________________________________________________________

In [76]:
model.fit([x_train,x_heading_train],y_train,
          epochs=5, batch_size=64,          
          validation_data=([x_val,x_heading_val], y_val),
         verbose=1)

Epoch 1/5
625/625 [==============================] - 1589s 3s/step - loss: 0.7981 - accuracy: 0.7316 - val_loss: 0.7806 - val_accuracy: 0.7351
Epoch 2/5
625/625 [==============================] - 1615s 3s/step - loss: 0.7693 - accuracy: 0.7366 - val_loss: 0.7627 - val_accuracy: 0.7386
Epoch 3/5
625/625 [==============================] - 1606s 3s/step - loss: 0.7537 - accuracy: 0.7415 - val_loss: 0.7489 - val_accuracy: 0.7427
Epoch 4/5
625/625 [==============================] - 1608s 3s/step - loss: 0.7457 - accuracy: 0.7428 - val_loss: 0.7385 - val_accuracy: 0.7454
Epoch 5/5
625/625 [==============================] - 1621s 3s/step - loss: 0.7287 - accuracy: 0.7460 - val_loss: 0.7230 - val_accuracy: 0.7492


In [79]:
val_loss,val_accuracy = model.evaluate([x_val,x_heading_val], y_val, verbose = 2)
print("val_loss:", round(val_loss,4)*100,"%")
print("val_accuracy:", round(val_accuracy,4)*100,"%")

313/313 - 99s - loss: 0.7234 - accuracy: 0.7492
val_loss: 72.34 %
val_accuracy: 74.92 %


## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)